In [ ]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import os
import arcpy

<h3>Enrutamiendo de directorios</h3>

In [ ]:
RUTA_ARCHIVO_ASIGNACION_RECONOCIMIENTO = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoReconocimiento\2.Asignacion_Vs_ObservacionesJuridicas\zInsumos\asignacion_general_marialabaja_estandarizada.xlsx"
RUTA_BDCONSOLIDADA = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb"
CAPA_OBSERVACION_JURIDICA = 'Analitica_UT_Consolidada\R_TERRENO'
RUTA_OBSERVACIONES_JURIDICAS = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoReconocimiento\2.Asignacion_Vs_ObservacionesJuridicas\zInsumos\Asignacion_Reconocimiento_Normalizado.xlsx"

CAPA_TERRENOS_R = 'Analitica_UT_Consolidada\R_TERRENO'
CAPA_TERRENOS_U = 'Analitica_UT_Consolidada\\U_TERRENO'
SHP = 'Asignacion_Terrenos_Con_Observaciones_Juridicas.shp'

ruta_shp = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoReconocimiento\2.Asignacion_Vs_ObservacionesJuridicas\zShp"

In [ ]:
df_asignacion_reconocimiento = pd.read_excel(RUTA_ARCHIVO_ASIGNACION_RECONOCIMIENTO, sheet_name='Normalizacion')
df_observaciones_juridicas = pd.read_excel(RUTA_OBSERVACIONES_JURIDICAS, sheet_name='Asignacion_E')

<h3>Llamada a resultados de la espacialización de las Observaciones</h3>

<h3>Unificación de Asignación y Objervaciones Jurídicas</h3>

In [ ]:

# ** Terrenos Gestionados SIG Espacializados
df_asignacion_observaciones = pd.merge(df_asignacion_reconocimiento, df_observaciones_juridicas, left_on='codigo', right_on='codigo_terreno', how='inner')

In [ ]:

# ** Estandarizacion Capitalize
df_asignacion_observaciones['reconocedor'] = df_asignacion_observaciones['reconocedor'].str.title()

In [ ]:

capa_terrenos_rurales = os.path.join(RUTA_BDCONSOLIDADA, CAPA_TERRENOS_R)
capa_terrenos_urbanos = os.path.join(RUTA_BDCONSOLIDADA, CAPA_TERRENOS_U)

# ** Terrenos Rurales
df_terreno_rural = pd.DataFrame.spatial.from_featureclass(capa_terrenos_rurales)

# ** Terrenos Urbanos
df_terreno_urbano = pd.DataFrame.spatial.from_featureclass(capa_terrenos_urbanos)

# ** Selección de columnas
columnas_selecciones = ['CODIGO',
    'CODIGO_ANTERIOR',
    'SHAPE']

# ** DF Terrenos Rurales, Urbanos
df_terreno_rural_reducido = df_terreno_rural[columnas_selecciones]
df_terreno_urbano_reducido = df_terreno_urbano[columnas_selecciones]

# ** Concatenación de dataframes
df_terrenos = pd.concat([df_terreno_rural_reducido, df_terreno_urbano_reducido])

In [ ]:

# ** Terrenos Gestionados SIG Espacializados
df_terreno_asignacion_observaciones = pd.merge(df_terrenos, df_asignacion_observaciones, left_on='CODIGO', right_on='codigo', how='inner')

In [ ]:

# ** Exportación a BD
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{'terrenos_asignados_con_obsjuridica'.upper()}"
ruta_salida_terrenos_asignacion_obsjuridica = os.path.join(RUTA_BDCONSOLIDADA, NOMBRE_CAPA_SALIDA)

df_terreno_asignacion_observaciones.spatial.to_featureclass(location=ruta_salida_terrenos_asignacion_obsjuridica)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA}")

In [ ]:

# ** Exportación SHP
ruta_salida_shp = os.path.join(ruta_shp, SHP)
df_terreno_asignacion_observaciones.spatial.to_featureclass(location=ruta_salida_shp)
print(f"Se exporta la capa {SHP}")